# 🏆 Gold Layer - KPIs e Agregações CNES
 
## **Objetivo**: Criar datasets analíticos prontos para consumo
 
## 📊 Agregações:
### 1. **Cobertura geográfica** - Estabelecimentos por região/UF/município
### 2. **Análise de capacidade** - Distribuição de serviços
### 3. **Indicadores de qualidade** - Scores e classificações
### 4. **Análise de complexidade** - Estrutura da rede de saúde
### 5. **Geolocalização** - Dados para mapas interativos

## 1️⃣ Setup

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime
import time

print("✅ Bibliotecas importadas!")

In [0]:
catalog_name = "datasus_project"
schema_silver = "silver"
schema_gold = "gold"

spark.sql(f"USE CATALOG {catalog_name}")
print(f"✅ Catalog: {catalog_name}")

## 2️⃣ Ler Dados Silver

In [0]:
# Ler tabela Silver
df_silver = spark.table(f"{catalog_name}.{schema_silver}.cnes_estabelecimentos_clean")

print("✅ Dados Silver carregados!")
print(f"   Registros: {df_silver.count():,}")

# View Temporária para Performance
df_silver.createOrReplaceTempView("silver_cnes_estabelecimentos")

## 3️⃣ Agregação 1 - Cobertura Geográfica por Região


In [0]:
print("📍 CRIANDO: Agregação por Região")
print("="*70)

# Agregação por região
df_gold_regiao = df_silver.groupBy("REGIAO") \
    .agg(
        F.count("*").alias("total_estabelecimentos"),
        F.countDistinct("UF_SIGLA").alias("total_estados"),
        F.countDistinct("CO_IBGE").alias("total_municipios"),
        
        # Totais por complexidade
        F.sum(F.when(F.col("COMPLEXIDADE") == "ATENÇÃO BÁSICA", 1).otherwise(0)).alias("total_atencao_basica"),
        F.sum(F.when(F.col("COMPLEXIDADE") == "MÉDIA COMPLEXIDADE", 1).otherwise(0)).alias("total_media_complexidade"),
        F.sum(F.when(F.col("COMPLEXIDADE") == "ALTA COMPLEXIDADE", 1).otherwise(0)).alias("total_alta_complexidade"),
        
        # Totais por flags de serviços
        F.sum(F.when(F.col("FLAG_CENTRO_CIRURGICO") == True, 1).otherwise(0)).alias("total_centro_cirurgico"),
        F.sum(F.when(F.col("FLAG_CENTRO_OBSTETRICO") == True, 1).otherwise(0)).alias("total_centro_obstetrico"),
        F.sum(F.when(F.col("FLAG_CENTRO_NEONATAL") == True, 1).otherwise(0)).alias("total_centro_neonatal"),
        F.sum(F.when(F.col("FLAG_ATEND_HOSPITALAR") == True, 1).otherwise(0)).alias("total_atend_hospitalar"),
        
        # Médias
        F.round(F.avg("SCORE_CAPACIDADE"), 2).alias("score_medio_capacidade"),
        F.round(F.avg("score_qualidade"), 2).alias("score_medio_qualidade"),
        
        # Com localização válida
        F.sum(F.when(F.col("LOCALIZACAO_VALIDA") == True, 1).otherwise(0)).alias("total_com_localizacao")
    ) \
    .withColumn("percentual_com_localizacao", 
        F.round((F.col("total_com_localizacao") / F.col("total_estabelecimentos")) * 100, 2)
    ) \
    .withColumn("data_atualizacao", F.current_timestamp())

print("✅ Agregação por região criada!")
df_gold_regiao.show(truncate=False)

In [0]:
# Salvar
table_gold_regiao = f"{catalog_name}.{schema_gold}.kpi_estabelecimentos_por_regiao"

df_gold_regiao.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(table_gold_regiao)

print(f"✅ Salvo: {table_gold_regiao}")

## 4️⃣ Agregação 2 - Detalhamento por UF

In [0]:
print("\n📍 CRIANDO: Agregação por UF")
print("="*70)

df_gold_uf = df_silver.groupBy("REGIAO", "UF_SIGLA", "UF_NOME") \
    .agg(
        F.count("*").alias("total_estabelecimentos"),
        F.countDistinct("CO_IBGE").alias("total_municipios"),
        
        # Por complexidade
        F.sum(F.when(F.col("COMPLEXIDADE") == "ATENÇÃO BÁSICA", 1).otherwise(0)).alias("atencao_basica"),
        F.sum(F.when(F.col("COMPLEXIDADE") == "MÉDIA COMPLEXIDADE", 1).otherwise(0)).alias("media_complexidade"),
        F.sum(F.when(F.col("COMPLEXIDADE") == "ALTA COMPLEXIDADE", 1).otherwise(0)).alias("alta_complexidade"),
        
        # Por categoria de capacidade
        F.sum(F.when(F.col("CATEGORIA_CAPACIDADE") == "ALTA CAPACIDADE", 1).otherwise(0)).alias("alta_capacidade"),
        F.sum(F.when(F.col("CATEGORIA_CAPACIDADE") == "MÉDIA CAPACIDADE", 1).otherwise(0)).alias("media_capacidade"),
        F.sum(F.when(F.col("CATEGORIA_CAPACIDADE") == "BAIXA CAPACIDADE", 1).otherwise(0)).alias("baixa_capacidade"),
        
        # Serviços especializados
        F.sum(F.when(F.col("FLAG_CENTRO_CIRURGICO") == True, 1).otherwise(0)).alias("com_centro_cirurgico"),
        F.sum(F.when(F.col("FLAG_CENTRO_OBSTETRICO") == True, 1).otherwise(0)).alias("com_centro_obstetrico"),
        F.sum(F.when(F.col("FLAG_ATEND_HOSPITALAR") == True, 1).otherwise(0)).alias("com_atend_hospitalar"),
        F.sum(F.when(F.col("FLAG_ATEND_AMBULATORIAL") == True, 1).otherwise(0)).alias("com_atend_ambulatorial"),
        
        # Qualidade
        F.round(F.avg("score_qualidade"), 2).alias("score_medio_qualidade"),
        F.sum(F.when(F.col("CLASSIFICACAO_QUALIDADE") == "EXCELENTE", 1).otherwise(0)).alias("qualidade_excelente"),
        F.sum(F.when(F.col("CLASSIFICACAO_QUALIDADE") == "BOA", 1).otherwise(0)).alias("qualidade_boa"),
        
        # Localização
        F.sum(F.when(F.col("LOCALIZACAO_VALIDA") == True, 1).otherwise(0)).alias("com_localizacao_valida")
    ) \
    .withColumn("percentual_localizacao_valida",
        F.round((F.col("com_localizacao_valida") / F.col("total_estabelecimentos")) * 100, 2)
    ) \
    .withColumn("data_atualizacao", F.current_timestamp()) \
    .orderBy("REGIAO", "UF_SIGLA")

print("✅ Agregação por UF criada!")
df_gold_uf.show(10, truncate=False)

In [0]:
# Salvar
table_gold_uf = f"{catalog_name}.{schema_gold}.kpi_estabelecimentos_por_uf"

df_gold_uf.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(table_gold_uf)

print(f"✅ Salvo: {table_gold_uf}")


## 5️⃣ Agregação 3 - Ranking de Municípios

In [0]:
print("\n🏙️ CRIANDO: Top Municípios com mais estabelecimentos")
print("="*70)

df_gold_municipios = df_silver.groupBy(
    "CO_IBGE", "UF_SIGLA", "UF_NOME", "REGIAO"
) \
    .agg(
        F.count("*").alias("total_estabelecimentos"),
        F.sum(F.when(F.col("FLAG_ATEND_HOSPITALAR") == True, 1).otherwise(0)).alias("com_internacao"),
        F.sum(F.when(F.col("COMPLEXIDADE") == "ALTA COMPLEXIDADE", 1).otherwise(0)).alias("alta_complexidade"),
        F.round(F.avg("SCORE_CAPACIDADE"), 2).alias("score_medio_capacidade")
    ) \
    .withColumn("data_atualizacao", F.current_timestamp())

# Adicionar ranking
window_brasil = Window.orderBy(F.desc("total_estabelecimentos"))
window_uf = Window.partitionBy("UF_SIGLA").orderBy(F.desc("total_estabelecimentos"))

df_gold_municipios = df_gold_municipios \
    .withColumn("ranking_brasil", F.row_number().over(window_brasil)) \
    .withColumn("ranking_uf", F.row_number().over(window_uf))

# Top 50 municípios
df_gold_top_municipios = df_gold_municipios.filter(F.col("ranking_brasil") <= 50)

print("✅ Top 50 municípios:")
df_gold_top_municipios.show(20, truncate=False)

In [0]:
# Salvar ranking completo
table_gold_municipios = f"{catalog_name}.{schema_gold}.kpi_estabelecimentos_por_municipio"

df_gold_municipios.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .partitionBy("UF_SIGLA") \
    .saveAsTable(table_gold_municipios)

print(f"✅ Salvo: {table_gold_municipios}")

## 6️⃣ Agregação 4 - Análise por Tipo de Estabelecimento

In [0]:
print("\n🏥 CRIANDO: Análise por Tipo de Estabelecimento")
print("="*70)

df_gold_tipos = df_silver.groupBy("TIPO_ESTABELECIMENTO", "COMPLEXIDADE") \
    .agg(
        F.count("*").alias("total"),
        F.countDistinct("UF_SIGLA").alias("presente_em_ufs"),
        F.round(F.avg("SCORE_CAPACIDADE"), 2).alias("score_medio"),
        F.sum(F.when(F.col("FLAG_ATEND_HOSPITALAR") == True, 1).otherwise(0)).alias("com_internacao"),
        F.sum(F.when(F.col("LOCALIZACAO_VALIDA") == True, 1).otherwise(0)).alias("com_coordenadas")
    ) \
    .withColumn("percentual_com_coordenadas",
        F.round((F.col("com_coordenadas") / F.col("total")) * 100, 2)
    ) \
    .withColumn("data_atualizacao", F.current_timestamp()) \
    .orderBy(F.desc("total"))

print("✅ Análise por tipo criada!")
df_gold_tipos.show(20, truncate=False)

In [0]:
# Salvar
table_gold_tipos = f"{catalog_name}.{schema_gold}.kpi_por_tipo_estabelecimento"

df_gold_tipos.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(table_gold_tipos)

print(f"✅ Salvo: {table_gold_tipos}")

## 7️⃣ Agregação 5 - Dataset para Mapa (Geolocalização)

In [0]:
print("\n🗺️ CRIANDO: Dataset para visualização em mapa")
print("="*70)

# Filtrar apenas estabelecimentos com localização válida
df_gold_mapa = df_silver.filter(F.col("LOCALIZACAO_VALIDA") == True) \
    .select(
        "CO_CNES",
        "NO_FANTASIA_LIMPO",
        "TIPO_ESTABELECIMENTO",
        "COMPLEXIDADE",
        "UF_SIGLA",
        "REGIAO",
        "NU_LATITUDE",
        "NU_LONGITUDE",
        "FLAG_ATEND_HOSPITALAR",
        "FLAG_CENTRO_CIRURGICO",
        "FLAG_CENTRO_OBSTETRICO",
        "SCORE_CAPACIDADE",
        "CATEGORIA_CAPACIDADE",
        "CLASSIFICACAO_QUALIDADE"
    ) \
    .withColumn("data_atualizacao", F.current_timestamp())

total_mapa = df_gold_mapa.count()
print(f"✅ Estabelecimentos com coordenadas válidas: {total_mapa:,}")

# Amostra
df_gold_mapa.show(5, truncate=True)

In [0]:
# Salvar
table_gold_mapa = f"{catalog_name}.{schema_gold}.dataset_mapa_estabelecimentos"

df_gold_mapa.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .partitionBy("REGIAO") \
    .saveAsTable(table_gold_mapa)

print(f"✅ Salvo: {table_gold_mapa}")

## 8️⃣ Agregação 6 - KPIs Gerais (Dashboard Principal)

In [0]:
print("\n📊 CRIANDO: KPIs Gerais para Dashboard")
print("="*70)

# Calcular KPIs consolidados
kpis_gerais = df_silver.agg(
    F.count("*").alias("total_estabelecimentos"),
    F.countDistinct("UF_SIGLA").alias("total_ufs"),
    F.countDistinct("CO_IBGE").alias("total_municipios"),
    F.countDistinct("REGIAO").alias("total_regioes"),
    
    # Por complexidade
    F.sum(F.when(F.col("COMPLEXIDADE") == "ATENÇÃO BÁSICA", 1).otherwise(0)).alias("total_atencao_basica"),
    F.sum(F.when(F.col("COMPLEXIDADE") == "MÉDIA COMPLEXIDADE", 1).otherwise(0)).alias("total_media_complexidade"),
    F.sum(F.when(F.col("COMPLEXIDADE") == "ALTA COMPLEXIDADE", 1).otherwise(0)).alias("total_alta_complexidade"),
    
    # Serviços
    F.sum(F.when(F.col("FLAG_ATEND_HOSPITALAR") == True, 1).otherwise(0)).alias("total_com_internacao"),
    F.sum(F.when(F.col("FLAG_CENTRO_CIRURGICO") == True, 1).otherwise(0)).alias("total_com_cirurgia"),
    F.sum(F.when(F.col("FLAG_CENTRO_OBSTETRICO") == True, 1).otherwise(0)).alias("total_com_obstetricia"),
    F.sum(F.when(F.col("FLAG_CENTRO_NEONATAL") == True, 1).otherwise(0)).alias("total_com_neonatal"),
    
    # Qualidade
    F.round(F.avg("score_qualidade"), 2).alias("score_medio_qualidade"),
    F.round(F.avg("SCORE_CAPACIDADE"), 2).alias("score_medio_capacidade"),
    
    # Localização
    F.sum(F.when(F.col("LOCALIZACAO_VALIDA") == True, 1).otherwise(0)).alias("total_com_coordenadas")
).collect()[0]

# Converter para DataFrame
kpis_data = [(
    "GERAL",
    kpis_gerais["total_estabelecimentos"],
    kpis_gerais["total_ufs"],
    kpis_gerais["total_municipios"],
    kpis_gerais["total_regioes"],
    kpis_gerais["total_atencao_basica"],
    kpis_gerais["total_media_complexidade"],
    kpis_gerais["total_alta_complexidade"],
    kpis_gerais["total_com_internacao"],
    kpis_gerais["total_com_cirurgia"],
    kpis_gerais["total_com_obstetricia"],
    kpis_gerais["total_com_neonatal"],
    kpis_gerais["score_medio_qualidade"],
    kpis_gerais["score_medio_capacidade"],
    kpis_gerais["total_com_coordenadas"],
    round((kpis_gerais["total_com_coordenadas"] / kpis_gerais["total_estabelecimentos"]) * 100, 2),
    datetime.now()
)]

schema_kpis = StructType([
    StructField("escopo", StringType()),
    StructField("total_estabelecimentos", LongType()),
    StructField("total_ufs", LongType()),
    StructField("total_municipios", LongType()),
    StructField("total_regioes", LongType()),
    StructField("total_atencao_basica", LongType()),
    StructField("total_media_complexidade", LongType()),
    StructField("total_alta_complexidade", LongType()),
    StructField("total_com_internacao", LongType()),
    StructField("total_com_cirurgia", LongType()),
    StructField("total_com_obstetricia", LongType()),
    StructField("total_com_neonatal", LongType()),
    StructField("score_medio_qualidade", DoubleType()),
    StructField("score_medio_capacidade", DoubleType()),
    StructField("total_com_coordenadas", LongType()),
    StructField("percentual_com_coordenadas", DoubleType()),
    StructField("data_atualizacao", TimestampType())
])

df_kpis_gerais = spark.createDataFrame(kpis_data, schema_kpis)

print("✅ KPIs Gerais calculados!")
df_kpis_gerais.show(truncate=False)

In [0]:
# Salvar
table_kpis_gerais = f"{catalog_name}.{schema_gold}.kpis_gerais_dashboard"

df_kpis_gerais.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(table_kpis_gerais)

print(f"✅ Salvo: {table_kpis_gerais}")

## 9️⃣ Criar View Consolidada

In [0]:
print("\n📊 CRIANDO: View consolidada para BI")
print("="*70)

# Criar view que une informações principais
spark.sql(f"""
    CREATE OR REPLACE VIEW {catalog_name}.{schema_gold}.vw_analise_completa_cnes AS
    SELECT 
        s.CO_CNES,
        s.NO_FANTASIA_LIMPO as nome_fantasia,
        s.NO_RAZAO_SOCIAL as razao_social,
        s.UF_SIGLA as uf,
        s.UF_NOME as estado,
        s.REGIAO as regiao,
        s.TIPO_ESTABELECIMENTO as tipo,
        s.COMPLEXIDADE as complexidade,
        s.NATUREZA_ORGANIZACAO_DESC as natureza,
        s.CATEGORIA_CAPACIDADE as categoria_capacidade,
        s.SCORE_CAPACIDADE as score_capacidade,
        s.CLASSIFICACAO_QUALIDADE as qualidade,
        s.FLAG_ATEND_HOSPITALAR as tem_internacao,
        s.FLAG_CENTRO_CIRURGICO as tem_cirurgia,
        s.FLAG_CENTRO_OBSTETRICO as tem_obstetricia,
        s.FLAG_CENTRO_NEONATAL as tem_neonatal,
        s.FLAG_ATEND_AMBULATORIAL as tem_ambulatorio,
        s.LOCALIZACAO_VALIDA as tem_coordenadas_validas,
        s.NU_LATITUDE as latitude,
        s.NU_LONGITUDE as longitude,
        s.data_processamento_silver as data_processamento
    FROM {catalog_name}.{schema_silver}.cnes_estabelecimentos_clean s
    WHERE s.CLASSIFICACAO_QUALIDADE IN ('EXCELENTE', 'BOA')
""")

print("✅ View criada: vw_analise_completa_cnes")

# Testar view
spark.sql(f"SELECT * FROM {catalog_name}.{schema_gold}.vw_analise_completa_cnes LIMIT 5").show()

## 🔟 Otimizar Tabelas Gold

In [0]:
print("\n⚡ OTIMIZANDO TABELAS GOLD")
print("="*70)

gold_tables = [
    table_gold_regiao,
    table_gold_uf,
    table_gold_municipios,
    table_gold_tipos,
    table_gold_mapa,
    table_kpis_gerais
]

for table in gold_tables:
    try:
        spark.sql(f"OPTIMIZE {table}")
        print(f"✅ {table.split('.')[-1]} otimizada")
    except Exception as e:
        print(f"⚠️ Erro ao otimizar {table}: {e}")

print("\n✅ Otimização concluída!")


## 📊 Resumo Final e Estatísticas

In [0]:
print("\n" + "="*70)
print("📊 RESUMO DAS TABELAS GOLD CRIADAS")
print("="*70)

# Listar todas as tabelas gold
gold_tables_list = spark.sql(f"SHOW TABLES IN {catalog_name}.{schema_gold}").collect()

print(f"\n✅ Total de tabelas/views: {len(gold_tables_list)}\n")

for table in gold_tables_list:
    table_name = table.tableName
    table_type = "VIEW" if table.isTemporary else "TABLE"
    
    try:
        if table_type == "TABLE":
            count = spark.table(f"{catalog_name}.{schema_gold}.{table_name}").count()
            print(f"📊 {table_name:50s} | Registros: {count:>10,}")
        else:
            print(f"👁️  {table_name:50s} | VIEW")
    except:
        print(f"⚠️  {table_name:50s} | Erro ao contar")

In [0]:
# Registrar log
def log_pipeline_execution(pipeline_name, status, records=0, exec_time=0, error=None):
    schema_control = StructType([
        StructField("pipeline_name", StringType(), False),
        StructField("execution_date", TimestampType(), False),
        StructField("status", StringType(), False),
        StructField("records_processed", LongType(), True),
        StructField("execution_time_seconds", DoubleType(), True),
        StructField("error_message", StringType(), True)
    ])
    log_data = [(pipeline_name, datetime.now(), status, records, exec_time, error)]
    df_log = spark.createDataFrame(log_data, schema_control)
    df_log.write.format("delta").mode("append").saveAsTable(f"{catalog_name}.bronze.pipeline_control")

log_pipeline_execution("gold_aggregations_cnes", "SUCCESS", 0, 0)
print("\n✅ Pipeline registrado no log!")


## CONCLUSÃO

In [0]:
print("\n" + "="*70)
print("GOLD LAYER - CONCLUÍDA")
print("="*70)

print("\n✅ AGREGAÇÕES CRIADAS:")
print("   1️⃣ KPIs por Região (5 regiões)")
print("   2️⃣ Detalhamento por UF (27 estados)")
print("   3️⃣ Ranking de Municípios (todos + top 50)")
print("   4️⃣ Análise por Tipo de Estabelecimento")
print("   5️⃣ Dataset para Mapas (com coordenadas)")
print("   6️⃣ KPIs Gerais para Dashboard")
print("   7️⃣ View Consolidada para BI")

print("\n📊 MÉTRICAS DISPONÍVEIS:")
print("   • Total de estabelecimentos por região/UF")
print("   • Distribuição por complexidade")
print("   • Cobertura de serviços especializados")
print("   • Scores de capacidade e qualidade")
print("   • Rankings e comparativos")
print("   • Dados geoespaciais para visualização")

print("\n🎯 PRÓXIMOS PASSOS:")
print("   1. Conectar Power BI / Tableau")
print("   2. Criar dashboards interativos")
print("   3. Análises exploratórias com SQL")
print("   4. Exportar dados para apresentações")

print("\n🚀 SEU PROJETO ESTÁ COMPLETO!")
print("   Bronze ✅ → Silver ✅ → Gold ✅")

print("\n" + "="*70)

In [0]:
%sql
-- Ver KPIs gerais
SELECT * FROM datasus_project.gold.kpis_gerais_dashboard;

-- Top 10 UFs com mais estabelecimentos
SELECT UF_NOME, total_estabelecimentos 
FROM datasus_project.gold.kpi_estabelecimentos_por_uf
ORDER BY total_estabelecimentos DESC
LIMIT 10;

-- Estabelecimentos de alta complexidade por região
SELECT REGIAO, total_alta_complexidade
FROM datasus_project.gold.kpi_estabelecimentos_por_regiao;

-- View completa para análises
SELECT * FROM datasus_project.gold.vw_analise_completa_cnes
WHERE regiao = 'Nordeste' AND complexidade = 'ALTA COMPLEXIDADE';